# PyCaret Testing

Testing model creation with PyCaret package

In [1]:
import pandas as pd

In [2]:
# Load Data
train = pd.read_feather("/home/jovyan/work/ray_experiments/data/train_feat_1_baseline.feather")

In [3]:
train.columns

Index(['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4',
       'var_5', 'var_6', 'var_7',
       ...
       'count_var_195', 'in_test_var_195', 'count_var_196', 'in_test_var_196',
       'count_var_197', 'in_test_var_197', 'count_var_198', 'in_test_var_198',
       'count_var_199', 'in_test_var_199'],
      dtype='object', length=602)

In [42]:
micro_test = train[train.columns[1:50]]

In [43]:
micro_test.columns

Index(['target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6',
       'var_7', 'var_8', 'var_9', 'var_10', 'var_11', 'var_12', 'var_13',
       'var_14', 'var_15', 'var_16', 'var_17', 'var_18', 'var_19', 'var_20',
       'var_21', 'var_22', 'var_23', 'var_24', 'var_25', 'var_26', 'var_27',
       'var_28', 'var_29', 'var_30', 'var_31', 'var_32', 'var_33', 'var_34',
       'var_35', 'var_36', 'var_37', 'var_38', 'var_39', 'var_40', 'var_41',
       'var_42', 'var_43', 'var_44', 'var_45', 'var_46', 'var_47'],
      dtype='object')

In [44]:
micro_test.shape

(200000, 49)

Building PyCaret Experiment

In [45]:
from pycaret.classification import *

In [46]:
exp_clf101 = setup(data = micro_test, target = 'target', session_id=123, use_gpu=True)

,Description,Value
0,session_id,123
1,Target,target
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(200000, 49)"
5,Missing Values,False
6,Numeric Features,48
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


# Modelling with PyCaret

In [ ]:
best_model = compare_models()

IntProgress(value=0, description='Processing: ', max=79)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:28:34
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.9014,0.7242,0.0371,0.6290,0.0701,0.0595,0.1357,0.1220
ada,Ada Boost Classifier,0.9007,0.7213,0.0550,0.5379,0.0998,0.0829,0.1484,58.7700
lr,Logistic Regression,0.9006,0.7029,0.0108,0.7112,0.0214,0.0184,0.0793,25.4670
qda,Quadratic Discriminant Analysis,0.9006,0.7114,0.0385,0.5516,0.0720,0.0597,0.1262,0.5430
ridge,Ridge Classifier,0.8999,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1380
rf,Random Forest Classifier,0.8999,0.6914,0.0004,0.2500,0.0007,0.0006,0.0082,63.4210
knn,K Neighbors Classifier,0.8968,0.5338,0.0086,0.1768,0.0165,0.0072,0.0179,136.3940
svm,SVM - Linear Kernel,0.8789,0.0000,0.0731,0.3930,0.0744,0.0473,0.0747,6.8510
dt,Decision Tree Classifier,0.8216,0.5358,0.1784,0.1565,0.1667,0.0673,0.0675,30.5980


# XGB Model

In [ ]:
xgb_model = create_model('xgboost', fold = 5)

In [ ]:
# tune_model
tuned_xgb = tune_model(xgb_model, optimize = 'AUC')

In [ ]:
plot_model(xgb_model, plot = 'parameter')

# Light GBM 

In [ ]:
lightgbm_model = create_model('lightgbm', fold = 5)

In [ ]:
# tune_model
tuned_lightgbm = tune_model(lightgbm_model, optimize = 'AUC')

# Catboost

In [ ]:
catboost_model = create_model('catboost', fold = 5)

In [ ]:
tuned_catboost = tune_model(catboost_model, optimize = 'AUC')

# Bagging Tests

In [ ]:
bagged_xgb = ensemble_model(tuned_xgb, optimize = 'AUC')

In [ ]:
bagged_lightgbm = ensemble_model(tuned_lightgbm, optimize = 'AUC')

In [ ]:
bagged_catboost = ensemble_model(tuned_catboost, optimize = 'AUC')

# Boosting ensemble_model

In [ ]:
boosted_xgb = ensemble_model(tuned_xgb, method = 'Boosting', optimize = 'AUC')

In [ ]:
merge_boost = create_model('xgboost')

In [ ]:
blendy = blend_models(estimator_list = [bagged_xgb, bagged_lightgbm, bagged_catboost], 
                      optimize = 'AUC')

# Plot out models

In [ ]:
plot_model(blendy, plot = 'auc')

In [ ]:
plot_model(blendy, plot = 'pr')

In [ ]:
plot_model(tuned_lightgbm, plot='feature')